In [ ]:
import pandas as pd
import pyodbc
import PySimpleGUI as sg

# simple GUI
layout = [[sg.Text('Podaj daty poczatku i konca zakresu danych raporu.')],      
          [sg.Text('date_from', size=(10,1)), sg.Input(do_not_clear=True)], 
          [sg.Text('date_to', size=(10,1)), sg.Input(do_not_clear=True)], 
          [sg.RButton('Ok', size=(10,1))]]      

window = sg.Window('Raport Vetbarim_MARKETY_NOS').Layout(layout)     

while True:      
    event, values = window.Read()      
    if event is None or event == 'Ok':      
        break      
window.Close()

try:
	date_from = values[0]
	date_to = values[1]
	date_name = date_from[:6]
	sql_conn = pyodbc.connect('DRIVER=SQL Server;SERVER=172.16.0.4;UID=pukcez;PWD=pruk56;APP=Microsoft Office 2013;WSID=POZHAN25;DATABASE=gam_prod') 
	# MARKETY
	query_markety = """
			SELECT CASE
				WHEN ZZ.[Kod platnika] IN ('N03882','N04499','N04869','N05034','N05035','N05052','N05157','N05318') THEN 'AMAZON'
				WHEN ZZ.[Kod platnika] IN ('N06097','N06105') THEN 'AUCHAN'
				WHEN ZZ.[Kod platnika] IN ('N00668','N01316','N02178','N02205','N02274','N02451','N02452','N02608','N02611','N02616',
				'N02617','N02618','N02620','N02622','N02624','N02648','N02785','N03567','N03597','N03788','N03890','N03891','N03894',
				'N03895','N03899','N03900','N03901','N03902','N03903','N03978','N03979','N04001','N04108','N04317','N04455','N04596','N04693','N06103') THEN 'LECLERC'
				ELSE C.Name END AS 'Market', Z.[Nr] 'Nr Zapasu', Z.Nazwa 'Opis', -ZZ.Ilosc 'Ilosc', -ZZ.[Kwota kosztu] 'Kwota Kosztu [PLN]'
			FROM Zapas Z, ZapisZapasu ZZ, Customer C
			WHERE ZZ.[Kod sprzedawcy Response]='Markety' AND ZZ.[Typ zapisu]=0 AND ZZ.DateInvoice between '{}' and '{}' 
			AND Z.__recid=ZZ.__parid AND ZZ.[Kod platnika]=C.Id AND ZZ.Producent='Verbatim' AND C.Name<>'X-KOM' """.format(date_from, date_to)

	df_final_markety = pd.read_sql(query_markety, sql_conn)

	# MARKETY_NOS

	query_nos = """
			SELECT CASE
			WHEN ZZ.[Kod platnika] IN ('N03882','N04499','N04869','N05034','N05035','N05052','N05157','N05318') THEN 'AMAZON'
			WHEN ZZ.[Kod platnika] IN ('N06097','N06105') THEN 'AUCHAN'
			WHEN ZZ.[Kod platnika] IN ('N00668','N01316','N02178','N02205','N02274','N02451','N02452','N02608','N02611','N02616','N02617',
			'N02618','N02620','N02622','N02624','N02648','N02785','N03567','N03597','N03788','N03890','N03891','N03894','N03895','N03899',
			'N03900','N03901','N03902','N03903','N03978','N03979','N04001','N04108','N04317','N04455','N04596','N04693','N06103') THEN 'LECLERC'
			ELSE C.Name END AS 'Nazwa Nabywcy', Z.[Nr] 'Nr Zapasu' , Z.Nazwa 'Nazwa Zapasu', -ZZ.Ilosc 'Ilosc'
			FROM Zapas Z, ZapisZapasu ZZ, Customer C
			WHERE ZZ.[Kod sprzedawcy Response]='Markety' AND ZZ.[Typ zapisu]=0 AND ZZ.DateInvoice between '{}' and '{}' 
			AND Z.__recid=ZZ.__parid AND ZZ.[Kod platnika]=C.Id AND ZZ.Producent='Verbatim' AND C.Name<>'X-KOM' AND Z.[Grupa zapasów]='NOS' """.format(date_from, date_to)

	#get DataFrames
	df_sql_nos = pd.read_sql(query_nos, sql_conn)
	df_cennik = pd.read_excel('CennikVerbatim.xlsx') 

	#marge and groupby
	df_merged_nos = pd.merge(df_sql_nos, df_cennik, on='Nr Zapasu')
	df_merged_nos['Kwota Kosztu [EUR]'] = df_merged_nos['Ilosc'] * df_merged_nos['Koszt EUR']
	df_final_nos = df_merged_nos.groupby(['Nazwa Nabywcy','Nazwa Zapasu','Nr Zapasu','Nr Katalogowy','Koszt EUR']).sum()

	# TOP10

	query_top10 = """
		SELECT
			x.DateInvoice 'Data',
			x.Producent,
			x.[Nr zapasu] 'Nr Zapasu',
			Z.Opis 'Nazwa Zapasu',
			CASE WHEN x.[kod sprzedawcy Response]='EXPORT' THEN NULL ELSE x.[Kod platnika] END 'Nr Nabywcy',
			CASE WHEN x.[kod sprzedawcy Response]='EXPORT' THEN NULL ELSE C.Name END 'Nazwa Nabywcy',
			x.Ilosc 'Ilosc'
		FROM (
			SELECT
				ZZ.__recid,
				zz.DateInvoice,
				ZZ.Producent,
				ZZ.[Nr zapasu],
				-ZZ.Ilosc 'Ilosc',
				ZZ.[Kod platnika],
				ZZ.[Kod sprzedawcy Response]
			FROM ZapisZapasu ZZ
			JOIN Customer C on C.Id=ZZ.[Kod platnika]
			WHERE 
				ZZ.Producent='VERBATIM' AND
				ZZ.[Typ zapisu]=0 AND
				ZZ.Grupa like 'NOS%' AND
				ZZ.DateInvoice between '{0}' and '{1}' AND
				( ZZ.[Kod platnika] in (SELECT TOP 10 ZZ.[Kod platnika] 'Kod platnika' FROM ZapisZapasu ZZ
										JOIN Customer C on C.Id=ZZ.[Kod platnika]
										WHERE
											ZZ.Producent='VERBATIM' AND
											ZZ.DateInvoice between '{0}' and '{1}' AND
											ZZ.Grupa like 'NOS%' AND
											( C.Response not in ('MARKETY','EXPORT') OR C.Id='N04064')
										GROUP BY ZZ.[Kod platnika], C.Name
										ORDER BY SUM(zz.[Kwota kosztu])))

			UNION

			SELECT
				ZZ.__recid,
				zz.DateInvoice,
				ZZ.Producent,
				ZZ.[Nr zapasu],
				-ZZ.Ilosc 'Ilosc',
				ZZ.[Kod platnika],
				ZZ.[Kod sprzedawcy Response]
			FROM ZapisZapasu ZZ
			JOIN Customer C on C.Id=ZZ.[Kod platnika]
			WHERE 
				ZZ.Producent='VERBATIM' AND
				ZZ.[Typ zapisu]=0 AND
				ZZ.Grupa like 'NOS%' AND
				ZZ.DateInvoice between '{0}' and '{1}' AND
				ZZ.[Kod sprzedawcy Response]='EXPORT'
			) X
		JOIN Zapas Z ON Z.Nr=X.[Nr zapasu]
			LEFT JOIN Customer C on C.id=x.[Kod platnika] """.format(date_from, date_to)

	#get DataFrames
	df_sql_top10 = pd.read_sql(query_top10, sql_conn)

	#handle NaN
	df_cennik.dropna(inplace=True)
	df_sql_top10.fillna(method='bfill', inplace=True)

	#marge and groupby
	df_merged_top10 = pd.merge(df_sql_top10, df_cennik, on='Nr Zapasu')
	df_merged_top10['Kwota Kosztu'] = df_merged_top10['Ilosc'] * df_merged_top10['Koszt EUR']
	df_final_top10 = df_merged_top10.groupby(['Nazwa Nabywcy','Nazwa Zapasu','Nr Zapasu','Nr Katalogowy','Koszt EUR']).sum()


	# Excel Save
	excel = pd.ExcelWriter('Verbatim_MARKETY_{}.xlsx'.format(date_name))
	df_final_top10.to_excel(excel,'TOP10')
	df_final_markety.to_excel(excel,'Markety', index=None)
	df_final_nos.to_excel(excel,'Markety_NOS')
	df_cennik.to_excel(excel,'Cennik', index=None)
	excel.save()

	sg.Popup('Wygenerowano Pomyslnie plik {}'.format('Verbatim_MARKETY_{}.xlsx'.format(date_name)))
except:
	sg.Popup('Podczas generowania raportu wystąpił błąd!')